In [ ]:
# Source: https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset?resource=download

In [1]:
#Step 1: Convert CSV to XLSX and Load Data
#Convert the raw CSV to Excel format and do a quick inspection to ensure there are no data type issues (dates, numbers, etc.)
import pandas as pd

# Paths
input_csv = r"C:\Users\hp\Documents\Personal\Career\Portfolio\IBM HR Insights\Data\Raw\WA_Fn-UseC_-HR-Employee-Attrition.csv"
cleaned_xlsx = r"C:\Users\hp\Documents\Personal\Career\Portfolio\IBM HR Insights\Data\Cleaned\WA_Fn-UseC_-HR-Employee-Attrition_Cleaned.xlsx"

# Read CSV
df = pd.read_csv(input_csv)

# Quick inspect
print(df.info())
print(df.head())

# Save as XLSX
df.to_excel(cleaned_xlsx, index=False)
print("CSV converted to XLSX and saved.")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

---

In [2]:
#Step 2: Data Profiling & Basic Cleaning
#Before any deep analysis, we need to:
#Remove/flag useless columns
#Check for any missing/irrelevant values
#Add a calculated “Tenure” field (YearsAtCompany is provided, but let’s double-check)
#Identify columns to drop (e.g., EmployeeCount, StandardHours—all same value)
#Make sure string/categorical columns are properly formatted

import pandas as pd

# Load cleaned Excel
file_path = r"C:\Users\hp\Documents\Personal\Career\Portfolio\IBM HR Insights\Data\Cleaned\WA_Fn-UseC_-HR-Employee-Attrition_Cleaned.xlsx"
df = pd.read_excel(file_path)

# 1. Check for columns with only one unique value
single_value_cols = [col for col in df.columns if df[col].nunique() == 1]
print("Columns with single value:", single_value_cols)

# 2. Check for missing values
missing = df.isnull().sum()
print("Missing values per column:\n", missing[missing > 0])

# 3. Check categorical columns for weird/duplicate values
categorical = df.select_dtypes(include='object')
for col in categorical.columns:
    print(f"\nUnique values in {col}: {df[col].unique()}")

# 4. Add Tenure (redundant but for practice)
df['Tenure'] = df['YearsAtCompany']

# 5. Drop useless columns
drop_cols = ['EmployeeCount', 'StandardHours', 'Over18']  # All have one value
df = df.drop(columns=drop_cols)

# 6. Save cleaned file
output_path = r"C:\Users\hp\Documents\Personal\Career\Portfolio\IBM HR Insights\Data\Cleaned\WA_Fn-UseC_-HR-Employee-Attrition_Cleaned_Final.xlsx"
df.to_excel(output_path, index=False)
print("Cleaned file saved:", output_path)


Columns with single value: ['EmployeeCount', 'Over18', 'StandardHours']
Missing values per column:
 Series([], dtype: int64)

Unique values in Attrition: ['Yes' 'No']

Unique values in BusinessTravel: ['Travel_Rarely' 'Travel_Frequently' 'Non-Travel']

Unique values in Department: ['Sales' 'Research & Development' 'Human Resources']

Unique values in EducationField: ['Life Sciences' 'Other' 'Medical' 'Marketing' 'Technical Degree'
 'Human Resources']

Unique values in Gender: ['Female' 'Male']

Unique values in JobRole: ['Sales Executive' 'Research Scientist' 'Laboratory Technician'
 'Manufacturing Director' 'Healthcare Representative' 'Manager'
 'Sales Representative' 'Research Director' 'Human Resources']

Unique values in MaritalStatus: ['Single' 'Married' 'Divorced']

Unique values in Over18: ['Y']

Unique values in OverTime: ['Yes' 'No']
Cleaned file saved: C:\Users\hp\Documents\Personal\Career\Portfolio\IBM HR Insights\Data\Cleaned\WA_Fn-UseC_-HR-Employee-Attrition_Cleaned_Final.

In [ ]:
#Step 3: Define and Add Key Metrics in Python
#We need to add calculated columns that directly answer the business questions
#1. Attrition Rate, by Group
#2. Pay Gap Analysis
#3. Burnout Risk
#4. Diversity Trend
#5. High Potential/Promotion Readiness



In [3]:
import pandas as pd

# Load cleaned file
file_path = r"C:\Users\hp\Documents\Personal\Career\Portfolio\IBM HR Insights\Data\Cleaned\WA_Fn-UseC_-HR-Employee-Attrition_Cleaned_Final.xlsx"
df = pd.read_excel(file_path)

# 1. Pay Gap - add avg income by role and gender
df['Role_Gender_AvgIncome'] = df.groupby(['JobRole', 'Gender'])['MonthlyIncome'].transform('mean')
df['Role_Gender_AvgIncome_Diff'] = df['MonthlyIncome'] - df['Role_Gender_AvgIncome']

# 2. Burnout Risk
df['BurnoutRisk'] = ((df['OverTime'] == 'Yes') & ((df['JobSatisfaction'] <= 2) | (df['WorkLifeBalance'] <= 2)))

# 3. Promotion Readiness
df['PromotionReady'] = (
    (df['PerformanceRating'] >= 3) &
    (df['YearsAtCompany'] >= 3) &
    (df['YearsSinceLastPromotion'] >= 2)
)

# Save output with specified sheet name
output_path = r"C:\Users\hp\Documents\Personal\Career\Portfolio\IBM HR Insights\Data\Output\HR_Attrition_Metrics.xlsx"
df.to_excel(output_path, index=False, sheet_name='HR_Attrition')
print("Metrics added. File saved:", output_path)


Metrics added. File saved: C:\Users\hp\Documents\Personal\Career\Portfolio\IBM HR Insights\Data\Output\HR_Attrition_Metrics.xlsx


In [4]:
#Step 4: Basic Analysis & Sanity Check
#Quickly profile the new metrics. This ensures:
#No calculation mistakes
#Distributions make sense
#You know what the data is saying before building visuals

import pandas as pd

file_path = r"C:\Users\hp\Documents\Personal\Career\Portfolio\IBM HR Insights\Data\Output\HR_Attrition_Metrics.xlsx"
df = pd.read_excel(file_path)

print("Attrition (Yes/No):")
print(df['Attrition'].value_counts())
print("\nBurnout Risk (True/False):")
print(df['BurnoutRisk'].value_counts())
print("\nPromotion Ready (True/False):")
print(df['PromotionReady'].value_counts())
print("\nPay Gap (Difference from Role-Gender Avg Income):")
print(df['Role_Gender_AvgIncome_Diff'].describe())
print("\nSample rows with Burnout Risk or Promotion Ready:")
print(df[df['BurnoutRisk'] | df['PromotionReady']][['EmployeeNumber', 'Attrition', 'BurnoutRisk', 'PromotionReady', 'MonthlyIncome', 'Gender', 'JobRole']].head(10))


Attrition (Yes/No):
Attrition
No     1233
Yes     237
Name: count, dtype: int64

Burnout Risk (True/False):
BurnoutRisk
False    1230
True      240
Name: count, dtype: int64

Promotion Ready (True/False):
PromotionReady
False    1038
True      432
Name: count, dtype: int64

Pay Gap (Difference from Role-Gender Avg Income):
count    1470.000000
mean        0.000000
std      2008.485278
min     -6165.327273
25%     -1209.930263
50%      -349.710526
75%      1142.467553
max      7005.689394
Name: Role_Gender_AvgIncome_Diff, dtype: float64

Sample rows with Burnout Risk or Promotion Ready:
    EmployeeNumber Attrition  BurnoutRisk  PromotionReady  MonthlyIncome  \
0                1       Yes         True           False           5993   
3                5        No        False            True           2909   
5                8        No        False            True           3068   
6               10        No         True           False           2670   
9               13        N